# 利用rating2inter.ipynb中U/I的index对features进行一一对应(meta-text)
- Reindex item feature ID with IDs generated in 0rating2inter.ipynb

In [1]:
import os
import pandas as pd

In [2]:
os.chdir('/home/bjf/bjf_projects/MMRec/data/pet')
os.getcwd()

'/home/bjf/bjf_projects/MMRec/data/pet'

In [3]:
# load item mapping
i_id_mapping = 'i_id_mapping.csv'
df = pd.read_csv(i_id_mapping, sep='\t')
print(f'shape: {df.shape}')
df[:4]

shape: (8510, 2)


,asin,itemID
0,1223000893,0
1,4847676011,1
2,9376674824,2
3,B00002N8FK,3


In [4]:
'从数据集meta文件中提取出U-I交互'
import gzip, json
meta_file = 'meta_Pet_Supplies.json.gz'# 定义meta文件路径

print('0 Extracting U-I interactions.')# 打印任务开始信息

def parse(path):#解析指定路径的压缩文件，并逐行生成文件内容。每行内容被解析为字典形式
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):#从指定路径的压缩文件中读取数据，并转换为DataFrame。
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF(meta_file)

print(f'Total records: {meta_df.shape}')
meta_df[:3]

0 Extracting U-I interactions.
Total records: (110707, 9)


,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,0615553605,"{'also_bought': ['0836269195', '1931993807', '...",Pet Qwerks Treat Cookbook with Cutters,10.95,{'Pet Supplies': 34134},http://ecx.images-amazon.com/images/I/41yroPoJ...,Pet Qwerks,"[[Pet Supplies, Dogs, Toys]]",Know exactly what your dog is eating with 101 ...
1,0615583474,"{'also_viewed': ['1937406679', '1936319284', '...",It's A Cat's Life,NaN,{'Pet Supplies': 452691},http://ecx.images-amazon.com/images/I/51%2BBp0...,NaN,[[Pet Supplies]],"An Interactive Cat Book, by Howie ""The World's..."
2,0972585419,"{'also_bought': ['B0002FP328', 'B0002FP32I', '...",Feathered Phonics The Easy Way To Teach Your B...,0.92,{'Pet Supplies': 21104},http://ecx.images-amazon.com/images/I/61tAtKN0...,Pet Media,"[[Pet Supplies, Pet Supplies: International Sh...",Volume 1: 96 Words & Phrases! This is the CD t...


In [5]:
# remapping
# 重新映射asin到itemID
# 创建一个字典，将asin映射到itemID
map_dict = dict(zip(df['asin'], df['itemID']))
# 使用字典将meta_df中的asin转换为itemID
meta_df['itemID'] = meta_df['asin'].map(map_dict)
meta_df.dropna(subset=['itemID'], inplace=True)# 删除那些映射后为NaN的行
meta_df['itemID'] = meta_df['itemID'].astype('int64')# 将itemID的数据类型转换为整型
#meta_df['description'] = meta_df['description'].fillna(" ")# 将description列中的NaN值填充为空格
meta_df.sort_values(by=['itemID'], inplace=True)# 按照itemID排序

print(f'shape: {meta_df.shape}')
meta_df[:2]

shape: (8510, 10)


,asin,related,title,price,salesRank,imUrl,brand,categories,description,itemID
14,1223000893,"{'also_bought': ['B000G1KNPK', 'B00BYHJ6UI', '...","Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",29.50,{'Pet Supplies': 39514},http://ecx.images-amazon.com/images/I/41yr4bsw...,NaN,"[[Pet Supplies, Cats, Toys]]",This combo pack provides you with all 3 Cat Si...,0
65,4847676011,"{'also_bought': ['B006YFZXGG', 'B002KMDNE6', '...",Natural Toothpaste - 2-1/2 oz.for dogs,6.83,{'Pet Supplies': 6424},http://ecx.images-amazon.com/images/I/41RnSg%2...,St. Jon,"[[Pet Supplies, Dogs, Health Supplies, Dental ...","In this day and age, we're more concerned and ...",1


In [6]:
ori_cols = meta_df.columns.tolist()# 将元数据 DataFrame 的列名转换为列表，以便后续操作

ret_cols = [ori_cols[-1]] + ori_cols[:-1]# 重新排列列名，将最后一个列名移到第一个，其余列名顺序后移
print(f'new column names: {ret_cols}')# 打印新的列名顺序

new column names: ['itemID', 'asin', 'related', 'title', 'price', 'salesRank', 'imUrl', 'brand', 'categories', 'description']


In [7]:
meta_df[:3]

,asin,related,title,price,salesRank,imUrl,brand,categories,description,itemID
14,1223000893,"{'also_bought': ['B000G1KNPK', 'B00BYHJ6UI', '...","Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",29.50,{'Pet Supplies': 39514},http://ecx.images-amazon.com/images/I/41yr4bsw...,NaN,"[[Pet Supplies, Cats, Toys]]",This combo pack provides you with all 3 Cat Si...,0
65,4847676011,"{'also_bought': ['B006YFZXGG', 'B002KMDNE6', '...",Natural Toothpaste - 2-1/2 oz.for dogs,6.83,{'Pet Supplies': 6424},http://ecx.images-amazon.com/images/I/41RnSg%2...,St. Jon,"[[Pet Supplies, Dogs, Health Supplies, Dental ...","In this day and age, we're more concerned and ...",1
96,9376674824,"{'also_bought': ['B000JMJ4CE', 'B003FVK47A', '...",Solid Gold S.E.P. (Stop Eating Poop) 3.5oz,10.77,{'Pet Supplies': 5295},http://ecx.images-amazon.com/images/I/41S0Ds8n...,Solid Gold,"[[Pet Supplies, Dogs, Health Supplies, Supplem...",Stop Eating Poop contains Glutamic Acid to det...,2


In [8]:
ret_df = meta_df[ret_cols]# 选取meta_df中需要的列，创建一个新的DataFrame ret_df
# dump
ret_df.to_csv(os.path.join('./', 'meta-Pet_Supplies.csv'), index=False)# 将ret_df DataFrame保存为CSV文件
print('done!')

done!


## Reload

In [9]:
indexed_df = pd.read_csv('meta-Pet_Supplies.csv')
print(f'shape: {indexed_df.shape}')
indexed_df[:4]

shape: (8510, 10)


,itemID,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,0,1223000893,"{'also_bought': ['B000G1KNPK', 'B00BYHJ6UI', '...","Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",29.50,{'Pet Supplies': 39514},http://ecx.images-amazon.com/images/I/41yr4bsw...,NaN,"[['Pet Supplies', 'Cats', 'Toys']]",This combo pack provides you with all 3 Cat Si...
1,1,4847676011,"{'also_bought': ['B006YFZXGG', 'B002KMDNE6', '...",Natural Toothpaste - 2-1/2 oz.for dogs,6.83,{'Pet Supplies': 6424},http://ecx.images-amazon.com/images/I/41RnSg%2...,St. Jon,"[['Pet Supplies', 'Dogs', 'Health Supplies', '...","In this day and age, we're more concerned and ..."
2,2,9376674824,"{'also_bought': ['B000JMJ4CE', 'B003FVK47A', '...",Solid Gold S.E.P. (Stop Eating Poop) 3.5oz,10.77,{'Pet Supplies': 5295},http://ecx.images-amazon.com/images/I/41S0Ds8n...,Solid Gold,"[['Pet Supplies', 'Dogs', 'Health Supplies', '...",Stop Eating Poop contains Glutamic Acid to det...
3,3,B00002N8FK,"{'also_bought': ['B000IGGFEQ', 'B000IGGH6W', '...",Heath Manufacturing S-1-8 Single Hanging Suet ...,1.77,"{'Patio, Lawn & Garden': 51680}",http://ecx.images-amazon.com/images/I/51JbAY9m...,Heath,"[['Pet Supplies', 'Birds', 'Feeding & Watering...",Squirrel resistant. Holds one Heath Suet Cake....


In [10]:
## Reload

i_uni = indexed_df['itemID'].unique()

print(f'# of unique items: {len(i_uni)}')

print('min/max of unique learners: {0}/{1}'.format(min(i_uni), max(i_uni)))

# of unique items: 8510
min/max of unique learners: 0/8509
